In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB
%matplotlib inline

In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

In [3]:
MAX_TRAINING = 10000
MAX_VALIDATION = 2000
MAX_TESTING = 2000
correct_class = {1 : 'airplane', 2 : 'automobile', 3 : 'bird', 4 : 'cat', 5 : \
                 'deer', 6 : 'dog', 7 : 'frog', 8 : 'horse', 9 : 'ship', 10 : 'truck'}

In [4]:
#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()

# Keeping the data between 0 and 1
x_train_max, x_test_max = x_train / 255.0, x_test / 255.0
y_train_max = to_categorical(y_train, num_classes=10)
y_test_max = to_categorical(y_test, num_classes=10)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, MAX_TRAINING*3, MAX_TRAINING)
x_val, y_val = get_random_data(x_train_max, y_train_max, MAX_TRAINING*3+1, x_train_max.shape[0], MAX_VALIDATION)
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=10000, Y=10000
Number of Validation Samples: X=2000, Y=2000
Number of Test Samples: X=2000, Y=2000


In [5]:
class MetricsAfterEachEpoch(tf.keras.callbacks.Callback):
    def on_train_begin(self, scores={}):
        self.loss, self.accuracy, self.mae, self.mse = [], [], [], []
    def on_train_end(self, logs={}):
        return self.loss, self.accuracy, self.mae, self.loss
    def on_epoch_end(self, epoch, scores):
        self.loss.append(scores.get('loss'))
        self.accuracy.append(scores.get('acc'))
        self.mae.append(scores.get('mean_absolute_error'))
        self.loss.append(scores.get('mean_squared_error'))
        return

In [6]:
def LeNet5Classifier(activation_func):
    # Creating a LeNet5 Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

    # #Instantiating Layer 2
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

    # #Instantiating Layer 3
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))

    model.add(Flatten())

    #Instantiating Layer 4
    model.add(Dense(84, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [7]:
def LeNet5Evaluator(curr_optimizer, activation_func, curr_epochs=25):
    model = LeNet5Classifier(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, callbacks=[MetricsAfterEachEpoch()], batch_size=2048).history
    score = model.evaluate(x_test, y_test, batch_size=512)
    return score, history

In [ ]:
testing_optimizers = ['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh', 'relu']

for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}
        
        if optimizer == 'RAdam':
            score_returned, history = LeNet5Evaluator(RAdam(lr=0.0005), activation)
        else:
            score_returned, history = LeNet5Evaluator(optimizer, activation)
        
        #Test Side Stats
        loss[(optimizer, activation)] = score_returned[0]
        accuracy[(optimizer, activation)] = score_returned[1]
        mae[(optimizer, activation)] = score_returned[2]
        mse[(optimizer, activation)] = score_returned[3]
        
        #Training Side Stats
        train_loss_dict[(optimizer, activation)] = history.get('loss')
        train_accuracy_dict[(optimizer, activation)] = history.get('acc')
        train_mae_dict[(optimizer, activation)] = history.get('mean_absolute_error')
        train_mse_dict[(optimizer, activation)] = history.get('mean_squared_error')
        
        bg = (loss, accuracy, mae, mse, train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict)
        #print (bg)
        writeToDB(optimizer, activation, 'LeNet5', bg, 'milestone-experiments', 'benchmark_testing')

Epoch 1/25
10000/10000 [==============================] - 4s 396us/step - loss: 2.3260 - acc: 0.0760 - mean_absolute_error: 0.1802 - mean_squared_error: 0.0904
Epoch 2/25
10000/10000 [==============================] - 4s 352us/step - loss: 2.3233 - acc: 0.0798 - mean_absolute_error: 0.1802 - mean_squared_error: 0.0903
Epoch 3/25
10000/10000 [==============================] - 4s 357us/step - loss: 2.3150 - acc: 0.0946 - mean_absolute_error: 0.1801 - mean_squared_error: 0.0902
Epoch 4/25
10000/10000 [==============================] - 4s 366us/step - loss: 2.3056 - acc: 0.1329 - mean_absolute_error: 0.1800 - mean_squared_error: 0.0900
Epoch 5/25
10000/10000 [==============================] - 4s 382us/step - loss: 2.2959 - acc: 0.1539 - mean_absolute_error: 0.1798 - mean_squared_error: 0.0899
Epoch 6/25
10000/10000 [==============================] - 5s 522us/step - loss: 2.2857 - acc: 0.1540 - mean_absolute_error: 0.1796 - mean_squared_error: 0.0896
Epoch 7/25
10000/10000 [================

In [17]:
loss

{('SGD', 'relu'): 2.298825853347778}

In [18]:
accuracy

{('SGD', 'relu'): 0.10449999964237214}

In [19]:
mae

{('SGD', 'relu'): 0.17991932642459868}

In [20]:
mse

{('SGD', 'relu'): 0.08992593061923981}

In [21]:
train_loss_dict

{('SGD', 'relu'): [2.303701672744751,
  2.303417487335205,
  2.303154260635376,
  2.3029071487426758,
  2.3026690887451173,
  2.3024488124847413,
  2.3022381484985353,
  2.302039360809326,
  2.301848092651367,
  2.3016646842956545,
  2.301492157363892,
  2.3013234283447264,
  2.3011592666625975,
  2.3009973583221437,
  2.300842265701294,
  2.300681693649292,
  2.3005234027862547,
  2.300366414642334,
  2.3002087245941163,
  2.300052359008789,
  2.2998901679992674,
  2.2997284362792967,
  2.29956358833313,
  2.299401658248901,
  2.299230917739868]}

In [22]:
train_accuracy_dict

{('SGD', 'relu'): [0.10169999967813492,
  0.10160000030994415,
  0.10170000064373017,
  0.10200000060796738,
  0.10189999989271165,
  0.10260000047683716,
  0.10279999934434891,
  0.10319999959468841,
  0.10330000057220459,
  0.10320000056028367,
  0.10250000060796738,
  0.10320000005960464,
  0.10319999997615814,
  0.10350000064373016,
  0.10420000056028367,
  0.10530000056028366,
  0.10519999947547913,
  0.10650000030994415,
  0.1064999994635582,
  0.1076999993443489,
  0.10900000005960464,
  0.10959999958276749,
  0.11080000013113021,
  0.11140000047683715,
  0.111899999833107]}

In [23]:
train_mae_dict

{('SGD', 'relu'): [0.18001309928894044,
  0.180007781624794,
  0.18000293180942536,
  0.17999832966327667,
  0.17999377522468568,
  0.17998968794345857,
  0.1799857249736786,
  0.17998195872306824,
  0.17997830407619475,
  0.17997479457855226,
  0.17997148869037627,
  0.17996826436519622,
  0.1799650949716568,
  0.17996194195747375,
  0.17995895643234253,
  0.17995582683086395,
  0.1799527683019638,
  0.17994969365596772,
  0.17994657549858092,
  0.17994339327812195,
  0.17994023158550262,
  0.1799369546175003,
  0.17993371980190276,
  0.17993046987056732,
  0.17992702305316924]}

In [24]:
train_mse_dict

{('SGD', 'relu'): [0.09002238937616348,
  0.09001674494743347,
  0.09001154217720032,
  0.09000665017366409,
  0.09000192174911499,
  0.08999757596254349,
  0.08999338257312775,
  0.08998945095539093,
  0.08998565465211868,
  0.08998200845718383,
  0.0899785859465599,
  0.0899752198934555,
  0.08997195633649827,
  0.08996872819662094,
  0.08996564069986343,
  0.08996243748664856,
  0.08995928218364715,
  0.08995614227056503,
  0.08995300195217133,
  0.08994987907409668,
  0.08994663865566253,
  0.08994342594146729,
  0.08994012947082519,
  0.08993689839839936,
  0.08993349393606186]}